<h2>Preparação dos dados</h2>
<h5>Wellington R Monteiro</h5>

<p>A intenção deste notebook é a de preparar e concatenar todos os datasets em uma base única para uma análise futura. Os mesmos datasets foram já analisados previamente dentro da pasta <em>data_engineer</em> onde as oportunidades de melhoria foram encontradas e mapeadas como, por exemplo, a normalização dos dados.</p>

<p>Como premissas foram usadas todas os datasets originais com a exceção do <em>olist_order_reviews_dataset.csv</em>, substituído pelo <em>olist_order_reviews_dataset_engineered.csv</em> gerado dentro da pasta <em>data_engineer</em>. Este arquivo foi então utilizado pelo notebook <b>1. Text Analysis</b> para gerar o dataset <em>reviews_comprehend.csv</em>.</p>

<p>A saída deste notebok é o <em>dataset_prepared.csv</em>.</p>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from pandas_profiling import ProfileReport

In [2]:
dataset_folder = 'datasets/'

In [3]:
pd.set_option('display.max_columns', None)

<h5>Limpando os datasets</h5>

In [4]:
df_geolocation = pd.read_csv(dataset_folder + 'olist_geolocation_dataset.csv', delimiter=',', encoding='utf-8')
df_geolocation['geolocation_zip_code_prefix'] = df_geolocation['geolocation_zip_code_prefix'].astype(str).str.zfill(5)
df_geolocation.drop_duplicates(subset='geolocation_zip_code_prefix', inplace=True)
df_geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
5,01012,-23.547762,-46.635361,são paulo,SP


In [5]:
df_customers = pd.read_csv(dataset_folder + 'olist_customers_dataset.csv', delimiter=',', encoding='utf-8')
df_customers['customer_zip_code_prefix'] = df_customers['customer_zip_code_prefix'].astype(str).str.zfill(5)
df_customers.drop(['customer_city', 'customer_state'], axis=1, inplace=True)
df_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056


In [6]:
df_sellers = pd.read_csv(dataset_folder + 'olist_sellers_dataset.csv', delimiter=',', encoding='utf-8')
df_sellers['seller_zip_code_prefix'] = df_sellers['seller_zip_code_prefix'].astype(str).str.zfill(5)
df_sellers.drop(['seller_city', 'seller_state'], axis=1, inplace=True)
df_sellers.head()

,seller_id,seller_zip_code_prefix
0,3442f8959a84dea7ee197c632cb2df15,13023
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031
3,c0f3eea2e14555b6faeea3dd58c1b1c3,04195
4,51a04a8a6bdcb23deccc82b0b80742cf,12914


In [7]:
df_products = pd.read_csv(dataset_folder + 'olist_products_dataset.csv', delimiter=',', encoding='utf-8')
df_products.drop(['product_name_lenght', 'product_description_lenght'], axis=1, inplace=True)
df_products.head()

,product_id,product_category_name,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,4.0,625.0,20.0,17.0,13.0


In [8]:
df_order_headers = pd.read_csv(dataset_folder + 'olist_orders_dataset.csv', delimiter=',', encoding='utf-8')
df_order_headers['order_purchase_timestamp'] = pd.to_datetime(df_order_headers['order_purchase_timestamp'].fillna('1900-01-01'))
df_order_headers['order_approved_at'] = pd.to_datetime(df_order_headers['order_approved_at'].fillna('1900-01-01'))
df_order_headers['order_delivered_carrier_date'] = pd.to_datetime(df_order_headers['order_delivered_carrier_date'].fillna('1900-01-01'))
df_order_headers['order_delivered_customer_date'] = pd.to_datetime(df_order_headers['order_delivered_customer_date'].fillna('1900-01-01'))
df_order_headers['order_estimated_delivery_date'] = pd.to_datetime(df_order_headers['order_estimated_delivery_date'].fillna('1900-01-01'))

df_order_headers['order_purchase_wday'] = df_order_headers['order_purchase_timestamp'].dt.weekday
df_order_headers['order_approval_wday'] = df_order_headers['order_approved_at'].dt.weekday
df_order_headers['order_delivery_carrier_wday'] = df_order_headers['order_delivered_carrier_date'].dt.weekday
df_order_headers['order_delivery_customer_wday'] = df_order_headers['order_delivered_customer_date'].dt.weekday
df_order_headers['order_delivery_estimate_wday'] = df_order_headers['order_estimated_delivery_date'].dt.weekday

df_order_headers['order_delivery_estimated_real_difference'] = (df_order_headers['order_delivered_customer_date'] - df_order_headers['order_estimated_delivery_date']).dt.days
df_order_headers['order_delivery_estimated_real_difference_excl_weekend'] = np.busday_count(df_order_headers['order_estimated_delivery_date'].dt.date, df_order_headers['order_delivered_customer_date'].dt.date)

df_order_headers['order_purchase_approval_difference'] = (df_order_headers['order_approved_at'] - df_order_headers['order_purchase_timestamp']).dt.days
df_order_headers['order_approval_carrier_difference'] = (df_order_headers['order_delivered_carrier_date'] - df_order_headers['order_approved_at']).dt.days
df_order_headers['order_approval_delivery_difference'] = (df_order_headers['order_estimated_delivery_date'] - df_order_headers['order_approved_at']).dt.days
                                                                                            
df_order_headers.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_purchase_wday,order_approval_wday,order_delivery_carrier_wday,order_delivery_customer_wday,order_delivery_estimate_wday,order_delivery_estimated_real_difference,order_delivery_estimated_real_difference_excl_weekend,order_purchase_approval_difference,order_approval_carrier_difference,order_approval_delivery_difference
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,0,0,2,1,2,-8,-6,0,2,15
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1,3,3,1,0,-6,-4,1,0,17
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,2,2,2,4,1,-18,-12,0,0,26
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,5,5,2,5,4,-13,-9,0,3,26
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1,1,2,4,0,-10,-6,0,0,12


In [9]:
df_order_items = pd.read_csv(dataset_folder + 'olist_order_items_dataset.csv', delimiter=',', encoding='utf-8')
df_order_items['shipping_limit_date'] = pd.to_datetime(df_order_items['shipping_limit_date'].fillna('1900-01-01'))
df_order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [10]:
df_order_payments = pd.read_csv(dataset_folder + 'olist_order_payments_dataset.csv', delimiter=',', encoding='utf-8')
df_methods_used = df_order_payments.groupby('order_id', as_index=False).count()[['order_id', 'payment_sequential']].rename(columns={'payment_sequential':'num_payment_methods_used'})
df_total_paid = df_order_payments.groupby('order_id', as_index=False).sum()[['order_id', 'payment_value']].rename(columns={'payment_value':'total_paid'})

df_order_payments = df_order_payments \
    .merge(df_methods_used, how='left', on='order_id') \
    .merge(df_total_paid, how='left', on='order_id')

df_payments_pivot = pd.pivot_table(df_order_payments, index='order_id', columns='payment_type', values=['payment_value', 'payment_installments'])

df_order_payments = df_payments_pivot \
    .merge(df_order_payments[['order_id', 'num_payment_methods_used', 'total_paid']], on='order_id')

C:\Users\wmont\Anaconda3\envs\keras\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [11]:
df_reviews = pd.read_csv(dataset_folder + 'reviews_comprehend.csv', delimiter=',', encoding='utf-8')
df_reviews['review_creation_date'] = pd.to_datetime(df_reviews['review_creation_date'].fillna('1900-01-01'))
df_reviews['review_answer_timestamp'] = pd.to_datetime(df_reviews['review_answer_timestamp'].fillna('1900-01-01'))
df_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,title_len,comment_len,TitleSentiment,TitleSentimentScore,ContentSentiment,ContentSentimentScore,ContentKeyEntities
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59,0,0,NaN,NaN,NaN,NaN,NaN
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13,0,0,NaN,NaN,NaN,NaN,NaN
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24,0,0,NaN,NaN,NaN,NaN,NaN
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,0,37,NaN,NaN,POSITIVE,0.834179,prazo
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,parabens lojas lannister adorei comprar pela i...,2018-03-01,2018-03-02 10:26:53,0,100,NaN,NaN,POSITIVE,0.951269,"internet,todos,e pratico,parabens,feliz pascoa..."


<h5>Concatenando os datasets</h5>

In [12]:
df = df_order_headers \
    .merge(df_reviews, how='left', on='order_id') \
    .merge(df_order_items, how='left', on='order_id') \
    .merge(df_customers, how='left', on='customer_id') \
    .merge(df_sellers, how='left', on='seller_id') \
    .merge(df_products, how='left', on='product_id') \
    .merge(df_geolocation.rename(columns={'geolocation_zip_code_prefix':'seller_zip_code_prefix', \
                                          'geolocation_lat':'seller_lat', 'geolocation_lng':'seller_lng', \
                                          'geolocation_city':'seller_city', 'geolocation_state':'seller_state'}), \
           how='left', on='seller_zip_code_prefix') \
    .merge(df_geolocation.rename(columns={'geolocation_zip_code_prefix':'customer_zip_code_prefix', \
                                          'geolocation_lat':'customer_lat', 'geolocation_lng':'customer_lng', \
                                          'geolocation_city':'customer_city', 'geolocation_state':'customer_state'}), \
           how='left', on='customer_zip_code_prefix') \
    .merge(df_order_payments, how='left', on='order_id') \
    .drop(['review_id', 'order_item_id'], axis=1)

In [13]:
df['customer_lat'].fillna(0, inplace=True)
df['customer_lng'].fillna(0, inplace=True)
df['seller_lat'].fillna(0, inplace=True)
df['seller_lng'].fillna(0, inplace=True)

df['distance_seller_customer'] = df.apply(lambda x: geodesic(tuple([x['customer_lat'], x['customer_lng']]), \
                     tuple([x['seller_lat'], x['seller_lng']])).km, axis=1)

In [14]:
df = df \
    .merge(df[['customer_id', 'price']].groupby('customer_id', as_index=False).sum() \
           .rename(columns={'price':'value_purchased_by_customer'}), how='left', on='customer_id') \
    .merge(df[['seller_id', 'price']].groupby('seller_id', as_index=False).sum() \
           .rename(columns={'price':'value_sold_by_seller'}), how='left', on='seller_id') \
    .merge(df[['product_id', 'price']].groupby('product_id', as_index=False).sum() \
           .rename(columns={'price':'value_sold_by_product'}), how='left', on='product_id') \
    .merge(df[['customer_id', 'price']].groupby('customer_id', as_index=False).count() \
           .rename(columns={'price':'qty_purchased_by_customer'}), how='left', on='customer_id') \
    .merge(df[['seller_id', 'price']].groupby('seller_id', as_index=False).count() \
           .rename(columns={'price':'qty_sold_by_seller'}), how='left', on='seller_id') \
    .merge(df[['product_id', 'price']].groupby('product_id', as_index=False).count() \
           .rename(columns={'price':'qty_sold_by_product'}), how='left', on='product_id')

In [15]:
df.to_csv(dataset_folder + 'dataset_prepared.csv', index=False, sep=',', encoding='utf-8')